In [6]:
import numpy as np
import pandas as pd
from pylab import plt, mpl
plt.style.use('ggplot')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(precision=5, suppress=True,formatter={'float': lambda x: f'{x:6.3f}'})
import yfinance as yf

In [7]:
df = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')[1]
df

Company Exchange Symbol                        Industry  \
0                   3M     NYSE    MMM                    Conglomerate   
1     American Express     NYSE    AXP              Financial services   
2                Amgen   NASDAQ   AMGN               Biopharmaceutical   
3               Amazon   NASDAQ   AMZN                       Retailing   
4                Apple   NASDAQ   AAPL          Information technology   
5               Boeing     NYSE     BA           Aerospace and defense   
6          Caterpillar     NYSE    CAT         Construction and mining   
7              Chevron     NYSE    CVX              Petroleum industry   
8                Cisco   NASDAQ   CSCO          Information technology   
9            Coca-Cola     NYSE     KO                  Drink industry   
10              Disney     NYSE    DIS  Broadcasting and entertainment   
11                 Dow     NYSE    DOW               Chemical industry   
12       Goldman Sachs     NYSE     GS              Financial services   
13          Home Depot     NYSE     HD                Home Improvement   
14           Honeywell   NASDAQ    HON                    Conglomerate   
15                 IBM     NYSE    IBM          Information technology   
16               Intel   NASDAQ   INTC          Semiconductor industry   
17   Johnson & Johnson     NYSE    JNJ         Pharmaceutical industry   
18      JPMorgan Chase     NYSE    JPM              Financial services   
19          McDonald's     NYSE    MCD                   Food industry   
20               Merck     NYSE    MRK         Pharmaceutical industry   
21           Microsoft   NASDAQ   MSFT          Information technology   
22                Nike     NYSE    NKE               Clothing industry   
23    Procter & Gamble     NYSE     PG      Fast-moving consumer goods   
24          Salesforce     NYSE    CRM          Information technology   
25           Travelers     NYSE    TRV                       Insurance   
26  UnitedHealth Group     NYSE    UNH             Managed health care   
27             Verizon     NYSE     VZ     Telecommunications industry   
28                Visa     NYSE      V              Financial services   
29             Walmart     NYSE    WMT                       Retailing   

    Date added                                  Notes Index weighting  
0   1976-08-09  As Minnesota Mining and Manufacturing           1.54%  
1   1982-08-30                                    NaN           3.64%  
2   2020-08-31                                    NaN           4.80%  
3   2024-02-26                                    NaN           2.93%  
4   2015-03-19                                    NaN           3.04%  
5   1987-03-12                                    NaN           3.36%  
6   1991-05-06                                    NaN           5.45%  
7   2008-02-19          Also 1930-07-18 to 1999-11-01           2.59%  
8   2009-06-08                                    NaN           0.81%  
9   1987-03-12          Also 1932-05-26 to 1935-11-20           1.02%  
10  1991-05-06                                    NaN           1.81%  
11  1991-05-06                                    NaN           0.94%  
12  2019-04-02                                    NaN           6.54%  
13  1999-11-01                                    NaN           6.23%  
14  2020-08-31             AlliedSignal and Honeywell           3.34%  
15  1979-06-29          Also 1932-05-26 to 1939-03-04           3.09%  
16  1999-11-01                                    NaN           0.72%  
17  1997-03-17                                    NaN           2.70%  
18  1991-05-06                                    NaN           3.07%  
19  1985-10-30                                    NaN           4.98%  
20  1979-06-29                                    NaN           2.16%  
21  1999-11-01                                    NaN           6.83%  
22  2013-09-20                                    NaN           

In [8]:
tickers = df.Symbol.to_list() + ["TSLA", "ORCL", "NFLX", "AAPL"]
tickers = list(set(tickers))
tickers

['CVX',
 'HON',
 'JPM',
 'MCD',
 'NFLX',
 'TRV',
 'VZ',
 'AMGN',
 'MRK',
 'NKE',
 'V',
 'DOW',
 'INTC',
 'BA',
 'HD',
 'WMT',
 'PG',
 'DIS',
 'CSCO',
 'AXP',
 'KO',
 'JNJ',
 'UNH',
 'AMZN',
 'CRM',
 'AAPL',
 'MMM',
 'CAT',
 'IBM',
 'ORCL',
 'TSLA',
 'MSFT',
 'GS']

In [52]:
start_date = '2023-01-27'
end_date = '2024-01-27'

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    data.to_csv(f'./data/daily/{ticker}.csv')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

## Technical Analysis

In [53]:
import pandas_ta as pta

In [54]:
def strategy_long(df):
    df["std_90"] = 1 + (((df["Close"] - df["Close"].shift(1)) / df["Close"].shift(1)).rolling(window=45).std())
    df["moving_average"] = df["Close"].rolling(window=17).mean()
    df["Open_prev"] = df["Open"].shift(1)
    df["std_90_prev"] = df["std_90"].shift(1)
    df["moving_average_prev"] = df["moving_average"].shift(1)
    df["Low_prev"] = df["Low"].shift(1)
    trades = df[(df["Open"] > df["moving_average_prev"]) & (df["Open"] < (df["Low_prev"] * df["std_90_prev"]))]
    trades["returns"] = (trades["Close"] - trades["Open"]) / trades["Open"]
    captial = 10000

    simulated_trades = []
    for index, row in trades.iterrows():
        captial = captial * (1 + row["returns"])
        simulated_trades.append([index,captial,row["returns"]])

    total_returns = (captial - 10000) / 10000
    drawdown = trades["returns"].min()
    sharpe = trades["returns"].mean() / trades["returns"].std() * np.sqrt(252)
    
    simulated_trades = pd.DataFrame(simulated_trades, columns=["Date", "Capital", "Returns"])
    simulated_trades.set_index("Date", inplace=True)

    return {
        "total_returns": total_returns,
        "drawdown": drawdown,
        "sharpe": sharpe,
        "simulated_trades": simulated_trades
    }

def strategy_long_short(df):
    df["std_90"] = 1 + (((df["Close"] - df["Close"].shift(1)) / df["Close"].shift(1)).rolling(window=90).std())
    df["moving_average"] = df["Close"].rolling(window=20).mean()
    df["Open_prev"] = df["Open"].shift(1)
    df["std_90_prev"] = df["std_90"].shift(1)
    df["moving_average_prev"] = df["moving_average"].shift(1)
    df["Low_prev"] = df["Low"].shift(1)
    df["High_prev"] = df["High"].shift(1)

    df["signal"] = np.where((df["Open"] > df["moving_average_prev"]) & (df["Open"] < (df["Low_prev"] * df["std_90_prev"])), 1, 0)
    df["signal"] = np.where((df["Open"] < df["moving_average_prev"]) & (df["Open"] > (df["High_prev"] * df["std_90_prev"])), -1, df["signal"])
    
    trades = df[df["signal"] != 0]
    trades["returns"] = ((trades["Close"] - trades["Open"]) / trades["Open"]) * trades["signal"]
    captial = 10000

    simulated_trades = []
    for index, row in trades.iterrows():
        captial = captial * (1 + row["returns"])
        simulated_trades.append([index,captial,row["returns"]])

    total_returns = (captial - 10000) / 10000
    drawdown = trades["returns"].min()
    sharpe = trades["returns"].mean() / trades["returns"].std() * np.sqrt(252)
    
    simulated_trades = pd.DataFrame(simulated_trades, columns=["Date", "Capital", "Returns"])
    simulated_trades.set_index("Date", inplace=True)

    return {
        "total_returns": total_returns,
        "drawdown": drawdown,
        "sharpe": sharpe,
        "simulated_trades": simulated_trades
    }

def visualize(trades, ticker):

    plt.figure(figsize=(10, 6))
    plt.plot(trades["Capital"])
    plt.title(f"Simulated Trades of {ticker}")
    plt.show()

In [55]:
main_df = pd.DataFrame()
sim_trades = dict()

for ticker in tickers:
    df = pd.read_csv(f'./data/daily/{ticker}.csv', index_col=0, parse_dates=True)
    trades = strategy_long(df)
    main_df[ticker] = [trades["total_returns"], trades["drawdown"], trades["sharpe"]]
    sim_trades[ticker] = trades["simulated_trades"]

main_df = main_df.T
main_df.columns = ["total_returns", "drawdown", "sharpe"]
    # visualize(trades, ticker)

C:\Users\ishaa\AppData\Local\Temp\ipykernel_6904\3210489495.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades["returns"] = (trades["Close"] - trades["Open"]) / trades["Open"]
C:\Users\ishaa\AppData\Local\Temp\ipykernel_6904\3210489495.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades["returns"] = (trades["Close"] - trades["Open"]) / trades["Open"]
C:\Users\ishaa\AppData\Local\Temp\ipykernel_6904\3210489495.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [56]:
main_df.sort_values(by="total_returns", ascending=False)

total_returns  drawdown    sharpe
MSFT       0.217940 -0.037147  2.778069
TSLA       0.209561 -0.059594  1.839127
AXP        0.186842 -0.025357  4.474939
CSCO       0.181713 -0.015904  4.227970
JPM        0.166019 -0.019183  3.731596
INTC       0.165538 -0.057846  1.610288
GS         0.151101 -0.016637  3.176101
ORCL       0.132463 -0.025032  2.189994
NKE        0.128994 -0.025678  2.377701
IBM        0.095376 -0.016904  2.515929
VZ         0.083426 -0.028952  1.725449
AMGN       0.078257 -0.029655  1.425109
CAT        0.077164 -0.050022  1.229822
KO         0.076254 -0.011947  2.906830
JNJ        0.075164 -0.019726  2.245856
AMZN       0.073485 -0.045508  0.809362
AAPL       0.068691 -0.029140  1.444779
TRV        0.062710 -0.026352  1.282968
WMT        0.056255 -0.014733  1.790385
NFLX       0.039243 -0.053410  0.527677
DOW        0.021490 -0.020181  0.743222
HON        0.019740 -0.018664  0.550221
MRK        0.015613 -0.022252  0.465724
MCD        0.014162 -0.019834  0.493507
V          0.011991 -0.021427  0.368203
BA         0.010245 -0.029787  0.329676
CRM        0.009655 -0.030255  0.268761
PG        -0.020361 -0.035362 -0.442757
MMM       -0.035444 -0.056841 -0.492489
HD        -0.041108 -0.024913 -0.766147
UNH       -0.088720 -0.028862 -2.174947
DIS       -0.139884 -0.032691 -3.515820
CVX       -0.162571 -0.025702 -4.135476

In [42]:
visualize(sim_trades["JPM"], "JPM")

In [20]:
sim_trades["TSLA"]

Capital   Returns
Date                              
2018-05-14   9625.807952 -0.037419
2018-06-05   9413.374228 -0.022069
2018-06-08   9373.832487 -0.004201
2018-06-11   9652.567524  0.029735
2018-06-13   9598.834941 -0.005567
...                  ...       ...
2024-02-28  46905.577717  0.008083
2024-02-29  46377.209316 -0.011265
2024-03-01  46867.531754  0.010572
2024-03-04  44370.037665 -0.053288
2024-03-18  43383.573226 -0.022233

[535 rows x 2 columns]